In [7]:
import bs4 as bs
import urllib.request
import time
from datetime import datetime
import pandas as pd
import numpy as np
import json
import os
import pickle

In [8]:
path_abs = os.path.abspath('.')

In [9]:
def load_MietspiegelInfo(path):
    df_metaInfos = pd.read_csv(path)
    df_metaInfos.rename(columns={'Vergleichsmiete für 65m²-Wohnungen*': 'Stadt', 
                            'Baujahr':'PLZ', 
                            '€/m²': 'Aktualität'}, inplace=True)
    return df_metaInfos

In [10]:
def load_MietspiegelURL(path):
    df_metaURLs = pd.read_csv(path)
    return df_metaURLs

In [11]:
def load_mietspiegel(url):
    data = urllib.request.urlopen(url)
    data = data.read()
    soup = bs.BeautifulSoup(data,'lxml')
    
    constYear = []
    sqrPrice = []
    counter = 0
    cut = None
    VergleichsmieteForflatSize = None
    df01 = None
    df02 = None
    comment = soup.find_all('div', {'class': 'InhaltContent'})
    
    if not comment:
        try:
            for idx,entry in enumerate(soup.find_all('td')):
                if counter == 0:
                    if entry.text.strip():
                        cut = int(idx/3)
                        if not VergleichsmieteForflatSize:
                            VergleichsmieteForflatSize = entry.text
                        continue
                if counter == 1:
                    constYear.append(entry.text)
                if counter == 2:
                    sqrPrice.append(entry.text.replace('\n',''))

                counter += 1
                if counter == 3:
                    counter = 0

            if len(sqrPrice) != len(constYear):
                print('Check this URL: ' + url)

            colNames = []
            for headline in soup.find_all('th'):
                colNames.append(headline.text)

            df01 = pd.DataFrame(columns = colNames)
            df01.iloc[:,1] = constYear[:cut]
            df01.iloc[:,2] = sqrPrice[:cut]

            colNames[0] = VergleichsmieteForflatSize
            df02 = pd.DataFrame(columns = colNames)
            df02.iloc[:,1] = constYear[cut:]
            df02.iloc[:,2] = sqrPrice[cut:]
        except:
            print('Attention: Here is a problem! URL: ' + url)
            comment = 'FEHLER'
    
    return df01, df02, comment

In [12]:
def load_mietspiegel_raw(url):
    data = urllib.request.urlopen(url)
    data = data.read()
    soup = bs.BeautifulSoup(data,'lxml')
    
    constYear = []
    sqrPrice = []
    counter = 0
    cut = None
    VergleichsmieteForflatSize = None
    df01 = None
    df02 = None
    
    for idx,entry in enumerate(soup.find_all('td')):
        if counter == 0:
            if entry.text.strip():
                cut = int(idx/3)
                if not VergleichsmieteForflatSize:
                    VergleichsmieteForflatSize = entry.text
                continue
        if counter == 1:
            constYear.append(entry.text)
        if counter == 2:
            sqrPrice.append(entry.text.replace('\n',''))

        counter += 1
        if counter == 3:
            counter = 0

    if len(sqrPrice) != len(constYear):
        print('Check this URL: ' + url)

    colNames = []
    for headline in soup.find_all('th'):
        colNames.append(headline.text)

    df01 = pd.DataFrame(columns = colNames)
    df01.iloc[:,1] = constYear[:cut]
    df01.iloc[:,2] = sqrPrice[:cut]

    colNames[0] = VergleichsmieteForflatSize
    df02 = pd.DataFrame(columns = colNames)
    df02.iloc[:,1] = constYear[cut:]
    df02.iloc[:,2] = sqrPrice[cut:]
    
    return df01, df02, comment

In [13]:
def executer(storage, df_metaInfos, df_metaURLs):
    df_comment = pd.DataFrame(df_metaURLs.shape[0] * [np.nan], columns = ['Bemerkung'])
    for idx,url in enumerate(df_metaURLs.URL):
        print(str(idx) + ': ' + url)
        storage[df_metaURLs.iloc[idx,0]] = []
        df01, df02, comment = load_mietspiegel(url)
        if df01 is not None:
            storage[df_metaURLs.iloc[idx,0]].append(df01) 
        if df02 is not None:
            storage[df_metaURLs.iloc[idx,0]].append(df02) 
        if comment:
            if type(comment) is str:
                df_comment.iloc[idx,0] = comment
            else:
                df_comment.iloc[idx,0] = comment[0].text
    df_metaInfos.insert(3, 'Bemerkung', df_comment.values)
    return df_metaInfos, storage
    

In [8]:
file_names_MetaInfo = ['/anwaltonlineMietspiegelInfo01.csv', 
                      '/anwaltonlineMietspiegelInfo02.csv', 
                      '/anwaltonlineMietspiegelInfo03.csv', 
                      '/anwaltonlineMietspiegelInfo04.csv', 
                      '/anwaltonlineMietspiegelInfo05.csv', 
                      '/anwaltonlineMietspiegelInfo06.csv', 
                      '/anwaltonlineMietspiegelInfo07.csv']

file_names_URL = ['/anwaltonlineMietspiegelURLs01.csv', 
                  '/anwaltonlineMietspiegelURLs02.csv', 
                  '/anwaltonlineMietspiegelURLs03.csv', 
                  '/anwaltonlineMietspiegelURLs04.csv', 
                  '/anwaltonlineMietspiegelURLs05.csv', 
                  '/anwaltonlineMietspiegelURLs06.csv', 
                  '/anwaltonlineMietspiegelURLs07.csv']

In [ ]:
storage = {}
df_metaInfosFinal = None
for i in range(7):
    path_metaInfos = path_abs + file_names_MetaInfo[i]
    df_metaInfosRaw = load_MietspiegelInfo(path_metaInfos)
    path_URLs = path_abs + file_names_URL[i]
    df_metaURLs = load_MietspiegelURL(path_URLs)
    
    df_metaInfos, storage = executer(storage, df_metaInfosRaw, df_metaURLs)
    if df_metaInfosFinal is None:
        df_metaInfosFinal = df_metaInfos
    else:
        df_metaInfosFinal = pd.concat([df_metaInfosFinal, df_metaInfos], axis=0)
        df_metaInfosFinal = df_metaInfosFinal.reset_index(drop=True)

In [35]:
df01, df02, comment = load_mietspiegel_raw('https://www.anwaltonline.com/mietrecht/mietspiegel/122/dinslaken')
storage['Dinslaken'] = [df01, df02]

## Cleaning Data

In [28]:
path_mietspiegelInfo = path_abs+'/metaInfosAll.csv'
df_mietspiegelLookUp = pd.read_csv(path_mietspiegelInfo, index_col=0)

In [27]:
df_mietspiegelLookUp

,Stadt,PLZ,Aktualität,Bemerkung
0,Aachen,52080,01.01.2017,NaN
1,Aalen,73434,01.02.2018,NaN
2,Abensberg,93326,01.01.2001,NaN
3,Ahaus,48683,01.01.2019,NaN
4,Ahlen,59227 - 59229,01.01.2010,NaN
5,Aichhalden,78733,01.01.2018,NaN
6,Albaxen,37671,NaN,Für Albaxen ist mit Abschlag von 5 Prozent der...
7,"Aldenhofen, Gemeinde",52457,NaN,Für die Gemeinde Aldenhoven gibt es keinen eig...
8,Allersberg,90584,NaN,FEHLER
9,Alpen,46519,01.01.2017,NaN


**2DO:** 
Aufspalten der Bindestrichpostleitzahlen in einzelne Postleitzahl

In [30]:
plz_with_ranges = df_mietspiegelLookUp.PLZ.loc[df_mietspiegelLookUp.PLZ.str.len() > 5]
for idx,plz_range in enumerate(plz_with_ranges):
    from_until = [int(x) for x in plz_range.split(' - ')]
    df_idx = int(plz_with_ranges.index[idx])
    current_row = df_mietspiegelLookUp.iloc[df_idx,:]
    current_row.PLZ = str(from_until[0])
    df_mietspiegelLookUp.iloc[df_idx, :] = current_row
    
df_mietspiegelLookUp.PLZ = df_mietspiegelLookUp.PLZ.astype('int')

In [17]:
df_mietspiegelLookUp.sort_values('PLZ')

,Stadt,PLZ,Aktualität,Bemerkung
145,Dresden,1067,01.01.2019,NaN
487,Radebeul,1445,01.12.2017,NaN
224,Großenhain,1558,01.01.2017,NaN
507,Riesa,1587,01.07.2016,NaN
125,Coswig (Sachsen),1640,01.01.2017,NaN
396,Meißen,1662,01.01.2013,NaN
479,Pirna,1796,01.01.2018,NaN
61,Bautzen,2625,01.09.2017,NaN
217,Görlitz,2826,01.09.1999,NaN
282,Hoyerswerda,2977,01.11.2008,NaN


In [133]:
path_mietspiegel_storage = path_abs+'/mietspiegelAll.p'
ms_storage = pickle.load( open( path_mietspiegel_storage, "rb" ) )

In [134]:
for key in ms_storage:
    if ms_storage[key]:
        print(key)
        for i in range(len(ms_storage[key])):
            try:
                price_current_city = ms_storage[key][i].iloc[:,2]
            except:
                print('WARNING! ERROR ' + key)
                continue
            low = []
            mid = []
            upper = []
            idx_nans = []
            for idx, mietpreis in enumerate(price_current_city):
                splitted = [x.replace(',','.') for x in list(mietpreis.split(' - '))]
                if len(splitted[0]) > 4:
                    splitted = [x.replace(',','.') for x in list(mietpreis.split(' – '))]
                try:
                    splitted = [float(x) for x in splitted]
                except:
                    pass
                if len(splitted) == 1:
                    low.append(np.NaN)
                    mid.append(splitted[0])
                    upper.append(np.NaN)
                if len(splitted) == 2:
                    low.append(splitted[0])
                    mid.append(np.NaN)
                    upper.append(splitted[1])
                if len(splitted) == 3:
                    low.append(splitted[0])
                    mid.append(splitted[1])
                    upper.append(splitted[2])
            ms_storage[key][i].insert(ms_storage[key][i].shape[1], 'low_Price', low)
            ms_storage[key][i].insert(ms_storage[key][i].shape[1], 'mid_Price', mid)
            ms_storage[key][i].insert(ms_storage[key][i].shape[1], 'upper_Price', upper)

Aachen
Aalen
Abensberg
Ahaus
Ahlen
Aichhalden
Allersberg
WARNING! ERROR Allersberg
Alpen
Alsdorf
Altdorf (Niederbayern)
Altdorf bei Nürnberg
Altdorf, Kreis Böblingen
Altena
Altenburg
Altenkirchen, Landkreis
Altötting
WARNING! ERROR Altötting
Altusried
WARNING! ERROR Altusried
Andernach
Angermünde
Ansbach
WARNING! ERROR Ansbach
Arnoldsweiler
Arnsberg
Arzberg
Aschaffenburg
Ascheberg
Aschersleben
Attendorn
Auerbach (Vogtland)
Augsburg
Aurich
Bad Alexandersbad
Bad Hindelang
WARNING! ERROR Bad Hindelang
Bad Kreuznach
Bad Laasphe
Bad Münstereifel
Bad Oeynhausen
Bad Salzuflen
Bad Sassendorf (Gemeinde)
Bad Waldsee
Bad Wurzach
Baden-Baden
WARNING! ERROR Baden-Baden
Balderschwang
WARNING! ERROR Balderschwang
Balve
Bamberg
Barsinghausen
Bautzen
Beckum
Bedburg
Bedburg-Hau
Belm
Bergen auf Rügen
Bergheim
Bergisch Gladbach
Bergkamen
Bergneustadt
Berlin
Bernau bei Berlin
Bernburg (Saale)
Bestwig
Betzigau
WARNING! ERROR Betzigau
Beverungen
Biberach an der Riß
Bielefeld
Bietigheim-Bissingen
Billerbeck
B

In [140]:
ms_storage['Jena'][0]

,Vergleichsmiete für 65m²-Wohnungen*,Baujahr,€/m²,low_Price,mid_Price,upper_Price
0,NaN,1905,"6,50 - 7,63 - 8,90",6.50,7.63,8.90
1,NaN,1925,"6,50 - 7,63 - 8,90",6.50,7.63,8.90
2,NaN,1955,"5,86 - 6,55 - 7,10",5.86,6.55,7.10
3,NaN,1965,"5,86 - 6,55 - 7,10",5.86,6.55,7.10
4,NaN,1975,"4,79 - 5,31 - 6,00",4.79,5.31,6.00
5,NaN,1985,"4,79 - 5,31 - 6,00",4.79,5.31,6.00
6,NaN,1995,"6,93 - 7,80 - 8,74",6.93,7.80,8.74
7,NaN,2000,"6,93 - 7,80 - 8,74",6.93,7.80,8.74
8,NaN,2005,"7,39 - 8,82 - 10,39",7.39,8.82,10.39
9,NaN,2010,"7,39 - 8,82 - 10,39",7.39,8.82,10.39


In [143]:
df_mietspiegelLookUp.to_csv(path_abs+'/metaInfosAllCleaned.csv')

In [145]:
#pickle.dump(storage, open( path_abs+"/mietspiegelAll.p", "wb" ) )
pickle.dump(ms_storage, open( path_abs+"/mietspiegelAllCleaned.p", "wb" ) )